In [48]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from pyspark.sql import SQLContext
from pyspark.sql.functions import col

In [2]:
spark = SparkContext(master='local', appName='Dataframes')
sqlContext = SQLContext(spark)

In [3]:
spark

<SparkContext master=local appName=Dataframes>

In [4]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [5]:
#aca cargo la ruta del archivo
path = "files/"

In [6]:
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line : line.split(','))

In [7]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

In [8]:
#sc.stop() #stop detiene el contexto

In [9]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

In [10]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
equiposOlimpicosRDD.map(lambda x: (x[2], x[1])).groupByKey()\
    .mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [12]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: 'ARG' in l)

In [13]:
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [14]:
equiposOlimpicosRDD.count()

1185

In [15]:
equiposOlimpicosRDD.countApprox(20)

1185

In [16]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [17]:
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

In [18]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [19]:
deportistaOlimpicoRDD.count()

135572

In [20]:
deportistaOlimpicoRDD.take(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['1', 'A Dijiang', '1', '24', '180', '80', '199']]

In [21]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [22]:
def eliminaEncabezado(indice, interador):
    return iter(list(interador)[1:])

In [23]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [24]:
deportistaOlimpicoRDD.take(5)

[['1', 'A Dijiang', '1', '24', '180', '80', '199'],
 ['2', 'A Lamusi', '1', '23', '170', '60', '199'],
 ['3', 'Gunnar Nielsen Aaby', '1', '24', '0', '0', '273'],
 ['4', 'Edgar Lindenau Aabye', '1', '34', '0', '0', '278'],
 ['5', 'Christine Jacoba Aaftink', '2', '21', '185', '82', '705']]

In [25]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: 
    (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
    )
)

In [26]:
deportistaOlimpicoRDD.take(5)

[(1, 'A Dijiang', 1, 24, 180, 80.0, 199),
 (2, 'A Lamusi', 1, 23, 170, 60.0, 199),
 (3, 'Gunnar Nielsen Aaby', 1, 24, 0, 0.0, 273),
 (4, 'Edgar Lindenau Aabye', 1, 34, 0, 0.0, 278),
 (5, 'Christine Jacoba Aaftink', 2, 21, 185, 82.0, 705)]

In [27]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False )
])

In [28]:
sqlContext.createDataFrame(deportistaOlimpicoRDD, schema).show(3)

+-------------+-------------------+------+----+------+----+---------+
|deportista_id|             nombre|genero|edad|altura|peso|equipo_id|
+-------------+-------------------+------+----+------+----+---------+
|            1|          A Dijiang|     1|  24|   180|80.0|      199|
|            2|           A Lamusi|     1|  23|   170|60.0|      199|
|            3|Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
+-------------+-------------------+------+----+------+----+---------+
only showing top 3 rows



In [29]:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [30]:
deportistaOlimpicoDF.show(3)

+-------------+-------------------+------+----+------+----+---------+
|deportista_id|             nombre|genero|edad|altura|peso|equipo_id|
+-------------+-------------------+------+----+------+----+---------+
|            1|          A Dijiang|     1|  24|   180|80.0|      199|
|            2|           A Lamusi|     1|  23|   170|60.0|      199|
|            3|Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
+-------------+-------------------+------+----+------+----+---------+
only showing top 3 rows



In [31]:
evento_schema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('evento', StringType(), False),
    StructField('deporte_id', IntegerType(), False)
])

In [32]:
eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

In [33]:
eventoDF.show(4)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [34]:
paises_schema = StructType([
    StructField("paises_id", IntegerType(), False),
    StructField("equipo", StringType(), False), 
    StructField("sigla", StringType(), False)
])


In [35]:
paisesDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(paises_schema).\
        load(path+"paises.csv")


In [36]:
paisesDF.show(4)

+---------+--------------------+-----+
|paises_id|              equipo|sigla|
+---------+--------------------+-----+
|        1|         30. Februar|  AUT|
|        2|A North American ...|  MEX|
|        3|           Acipactli|  MEX|
|        4|             Acturus|  ARG|
+---------+--------------------+-----+
only showing top 4 rows



In [37]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])


In [38]:
resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")


In [39]:
resultadosDF.show(4)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
+------------+-------+-------------+--------+---------+
only showing top 4 rows



In [40]:
deportes_schema = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False)
])

In [41]:
deportesDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(deportes_schema).\
            load(path+"juegos.csv")

In [42]:
deportesDF.show(2)

+----------+-----------+
|deporte_id|    deporte|
+----------+-----------+
|         1|1896 Verano|
|         2|1900 Verano|
+----------+-----------+
only showing top 2 rows



In [43]:
deportesDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [44]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [45]:
#eliminanado y renombrando columnas
deportistaOlimpicoDF = deportistaOlimpicoDF.withColumnRenamed('genero','sexo').drop('altura')

In [46]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [50]:
deportistaOlimpicoDF = deportistaOlimpicoDF.select('deportista_id', 'nombre', col('edad').alias('edadAlJugar'), 'equipo_id')

In [51]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [52]:
deportistaOlimpicoDF.sort('edadAlJugar').show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|          133|           Franz Abb|          0|      399|
|          167|Ould Lamine Abdallah|          0|      362|
|           66|     Mohamed Abakkar|          0|     1003|
|          163|     Ismail Abdallah|          0|     1095|
|          139|George Ioannis Abbot|          0|     1043|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [56]:
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(deportistaOlimpicoDF.edadAlJugar != 0)

In [57]:
deportistaOlimpicoDF.sort('edadAlJugar').show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        52070|        Etsuko Inada|         11|      514|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        47618|Sonja Henie Toppi...|         11|      742|
+-------------+--------------------+-----------+---------+
only showing top 5 rows

